# tokenizing non-parsed html
*note: below, classes (sponsored & non-sponsored) are represented by 1 and 0, respectively*

### load data into rdd and create parser

In [83]:
path_to_data = "/Users/kev/largeDataTeam/data/test2/*"
path_to_train = "/Users/kev/largeDataTeam/data/train.csv"

data = sc.wholeTextFiles(path_to_data)

### load training data labels into SFrame

In [61]:
import graphlab as gl
train = gl.SFrame(path_to_train)

PROGRESS: Finished parsing file /Users/kev/largeDataTeam/data/train.csv
PROGRESS: Parsing completed. Parsed 100 lines in 0.569421 secs.
------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[str,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------
PROGRESS: Finished parsing file /Users/kev/largeDataTeam/data/train.csv
PROGRESS: Parsing completed. Parsed 101107 lines in 0.26423 secs.


In [171]:
t = list(train['file'])
s = list(train['sponsored'])
ts = zip(train['file'],train['sponsored'])

### organize and map the data

In [168]:
mapped_data = data.map(lambda x: [x[0].split("/")[-1],
                                  x[1],
                                  "train" if x[0].split("/")[-1] in t else "test",
                                  1 if (x[0].split("/")[-1],1) in ts else 0])

mapped_train_data = mapped_data.filter(lambda x: x[2]=="train")
mapped_train_data_0 = mapped_data.filter(lambda x: x[3]==0)
mapped_train_data_1 = mapped_data.filter(lambda x: x[3]==1)

### adding features to the sframe

In [169]:
sf = gl.SFrame.from_rdd(mapped_data)
sf['file'] = sf['X1'].apply(lambda x: x[0])
sf['html'] = sf['X1'].apply(lambda x: x[1])
sf['train'] = sf['X1'].apply(lambda x: x[2])
sf['class'] = sf['X1'].apply(lambda x: x[3])

- preview sframe

In [170]:
sf

X1,file,html,train,class
"[1767762_raw_html.txt,<!DOCTYPE html>\n<htm ...",1767762_raw_html.txt,"<!DOCTYPE html>\n<html xmlns:fb=""http://ogp.me ...",train,1
"[3279553_raw_html.txt,<!DOCTYPE html>\n<!--[if ...",3279553_raw_html.txt,<!DOCTYPE html>\n<!--[iflt IE 7]&gt;&lt;html ...,train,0


### tokenize the html

In [197]:
mapped_tokens_0 = mapped_train_data_0.map(lambda x: x[1]).flatMap(lambda a: a.split(' ')).map(lambda word: (word, 1))
mapped_tokens_1 = mapped_train_data_1.map(lambda x: x[1]).flatMap(lambda a: a.split(' ')).map(lambda word: (word, 1))

from operator import add

tokens_0 = mapped_tokens_0.reduceByKey(add)
tokens_1 = mapped_tokens_1.reduceByKey(add)

- preview results of tokenization

In [198]:
print tokens_0.take(5)
print
print tokens_1.take(5)

[(u'', 12588), (u"$('#celebrity-menu').clone().appendTo('#navbar-subnav').attr('id',", 1), (u'all', 4), (u'src="/image-library/land/500/n/xnatalia-kills-credit-famous-1.jpg.pagespeed.ic.ESCoFQwb-D.jpg"/>\n', 1), (u'thumbnail">\n', 3)]

[(u'', 14819), (u'class="paragraph"', 64), (u'Edison', 2), (u'Van', 2), (u'president,', 1)]


### compare the tokens from class 1 and class 0
- tokens IN class 1 BUT NOT in class 0
- counts excluded

In [193]:
tokens_unique_to_class_1 = tokens_1.subtractByKey(tokens_0).map(lambda x: x[0])
tokens_unique_to_class_1.take(5)

[u'1,800', u'Edison', u'id="commentArea">\n', u'founder', u'every']

- tokens IN BOTH class 1 AND class 0
- counts excluded

In [190]:
intersect = tokens_0.map(lambda x: x[0]).intersection(tokens_1.map(lambda x: x[0]))
intersect.take(5)

[u'', u'<!DOCTYPE', u'go', u'When', u'with']

### put tokens into sframe

In [202]:
tokens_sf_0 = gl.SFrame.from_rdd(tokens_0.map(lambda x: x[0]))
tokens_sf_1 = gl.SFrame.from_rdd(tokens_1.map(lambda x: x[0]))

In [205]:
tokens_sf_0.print_rows(25)
tokens_sf_1.print_rows(25)

+--------------------------------+
|               X1               |
+--------------------------------+
|                                |
| $('#celebrity-menu').clone...  |
|              all               |
| src="/image-library/land/5...  |
|         thumbnail">\n          |
|    title="Entertainment">\n    |
|        width="125"/>\n         |
| t||(t=r),e.inPlace=s,e.fla...  |
|             skip:              |
|            comments            |
|            css:{\n             |
|          Exclusives\n          |
| information",this.sourceUR...  |
|        href="/tv/news/"        |
| src="/image-library/land/2...  |
|         title="rss">\n         |
| Date();a=s.createElement(o),\n |
|             Watch              |
|            finally             |
|          '_blank';\n           |
|     class="icon-gplus">\n      |
| Object.defineProperty(t,e,...  |
|      'value':Date.now()})      |
|     if(FamUserdata.country     |
|        'UA-1609907-1',         |
+-------------------